<a href="https://colab.research.google.com/github/kevin6449/ironman2024_genai/blob/main/gen_ai_day11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用 Gemini API 產生結構化輸出

Gemini 預設產生非結構化文本，但某些應用程式需要結構化文字。對於這些用例，您可以限制 Gemini 使用 JSON 回應，JSON 是一種適合自動處理的結構化資料格式。您也可以限制模型使用枚舉中指定的選項之一進行回應。

以下是一些可能需要模型結構化輸出的用例：

- 透過從報紙文章中提取公司資訊來建立公司資料庫。
- 從簡歷中提取標準化資訊。
- 從食譜中提取成分，並顯示每種成分的雜貨網站連結。

在提示中，您可以要求 Gemini 產生 JSON 格式的輸出，但請注意，該模型不保證會產生 JSON，並且只產生 JSON。為了獲得更具確定性的回應，您可以在 responseSchema 欄位中傳遞特定的 JSON 模式，以便 Gemini 始終以預期的結構進行回應。


如何使用 generateContent您選擇的 SDK 或直接使用 REST API 的方法來產生 JSON。
顯示了純文字輸入，儘管 Gemini 還可以對包含圖像、 視訊和音訊的多模式請求產生 JSON 回應。


In [1]:
import google.generativeai as genai

In [2]:
from google.colab import userdata

API_KEY=userdata.get('GOOGLE_API_KEY')

In [3]:
#genai.configure(api_key="YOUR_API_KEY")

# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

# 生成 JSON
當模型配置為輸出 JSON 時，它會使用 JSON 格式的輸出來回應任何提示。

您可以透過提供架構來控制 JSON 回應的結構。有兩種方法可以為模型提供架構：

- 作為提示中的文本
- 作為透過模型配置提供的結構化模式

### 在提示中提供模式作為文本

In [9]:
model = genai.GenerativeModel("gemini-1.5-pro-latest")
prompt = """以 JSON 格式列出一些好吃的中餐食譜。

使用此 JSON 架構：

Recipe = {'recipe_name': str, 'ingredients': list[str]}
Return: list[Recipe]"""
result = model.generate_content(prompt)
print(result)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n[\n  {\n    \"recipe_name\": \"\u5bae\u4fdd\u96de\u4e01\",\n    \"ingredients\": [\n      \"\u96de\u8089\u4e01\",\n      \"\u82b1\u751f\",\n      \"\u4e7e\u8fa3\u6912\",\n      \"\u82b1\u6912\",\n      \"\u8591\",\n      \"\u849c\",\n      \"\u8525\",\n      \"\u91ac\u6cb9\",\n      \"\u918b\",\n      \"\u7cd6\",\n      \"\u592a\u767d\u7c89\"\n    ]\n  },\n  {\n    \"recipe_name\": \"\u9ebb\u5a46\u8c46\u8150\",\n    \"ingredients\": [\n      \"\u5ae9\u8c46\u8150\",\n      \"\u8c6c\u7d5e\u8089\",\n      \"\u8c46\u74e3\u91ac\",\n      \"\u8fa3\u6912\u91ac\",\n      \"\u82b1\u6912\u7c89\",\n      \"\u8591\",\n      \"\u849c\",\n      \"\u8525\",\n      \"\u91ac\u6cb9\",\n      \"\u7cd6\",\n      \"\u592a\u767d\u7c89\"\n    ]\n  },\n  {\n    \"recipe_na

### 透過模型配置提供模式

In [8]:
import typing_extensions as typing

class Recipe(typing.TypedDict):
    recipe_name: str
    ingredients: list[str]

model = genai.GenerativeModel("gemini-1.5-pro-latest")
result = model.generate_content(
    "列出一些好吃的中餐食譜。",
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json", response_schema=list[Recipe]
    ),
)
print(result)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "[{\"ingredients\": [\"\u732a\u8089\", \"\u767d\u83dc\", \"\u8c46\u8150\", \"\u7c89\u6761\", \"\u6728\u8033\", \"\u8471\", \"\u59dc\", \"\u849c\", \"\u9171\u6cb9\", \"\u6599\u9152\", \"\u76d0\", \"\u7cd6\"], \"recipe_name\": \"\u767d\u83dc\u7096\u7c89\u6761\"}, {\"ingredients\": [\"\u9e21\u8089\", \"\u9752\u6912\", \"\u7ea2\u6912\", \"\u8471\", \"\u59dc\", \"\u849c\", \"\u8fa3\u6912\", \"\u82b1\u6912\", \"\u9171\u6cb9\", \"\u6599\u9152\", \"\u76d0\", \"\u7cd6\"], \"recipe_name\": \"\u5bab\u4fdd\u9e21\u4e01\"}, {\"ingredients\": [\"\u571f\u8c46\", \"\u9752\u6912\", \"\u7ea2\u6912\", \"\u8471\", \"\u59dc\", \"\u849c\", \"\u9171\u6cb9\", \"\u918b\", \"\u76d0\", \"\u7cd6\"], \"recipe_name\": \"\u571f\u8c46\u4e1d\"}, {\"ingredients\": [\"\u756a\u8304\", \"\u9e21\u

### 使用類型提示註解定義架構
定義模式最簡單的方法是使用類型提示註解。

In [11]:
generation_config={"response_mime_type": "application/json",
                   "response_schema": list[Recipe]}
print(result)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n[\n  {\n    \"recipe_name\": \"\u5bae\u4fdd\u96de\u4e01\",\n    \"ingredients\": [\n      \"\u96de\u8089\u4e01\",\n      \"\u82b1\u751f\",\n      \"\u4e7e\u8fa3\u6912\",\n      \"\u82b1\u6912\",\n      \"\u8591\",\n      \"\u849c\",\n      \"\u8525\",\n      \"\u91ac\u6cb9\",\n      \"\u918b\",\n      \"\u7cd6\",\n      \"\u592a\u767d\u7c89\"\n    ]\n  },\n  {\n    \"recipe_name\": \"\u9ebb\u5a46\u8c46\u8150\",\n    \"ingredients\": [\n      \"\u5ae9\u8c46\u8150\",\n      \"\u8c6c\u7d5e\u8089\",\n      \"\u8c46\u74e3\u91ac\",\n      \"\u8fa3\u6912\u91ac\",\n      \"\u82b1\u6912\u7c89\",\n      \"\u8591\",\n      \"\u849c\",\n      \"\u8525\",\n      \"\u91ac\u6cb9\",\n      \"\u7cd6\",\n      \"\u592a\u767d\u7c89\"\n    ]\n  },\n  {\n    \"recipe_na

## 使用列舉來限制輸出


In [13]:
!curl -o organ.jpg https://upload.wikimedia.org/wikipedia/commons/0/06/Orgel_i_Tranebjerg_Kirke_%28Sams%C3%B8_Kommune%29.JPG

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5864k  100 5864k    0     0   823k      0  0:00:07  0:00:07 --:--:--  933k


In [15]:
# Upload the file and print a confirmation.
sample_file = genai.upload_file(path="organ.jpg",
                            display_name="organ")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

Uploaded file 'organ' as: https://generativelanguage.googleapis.com/v1beta/files/t4gdqk6ltr5o


In [20]:
import google.generativeai as genai
!pip install typing_extensions

In [29]:
import enum
from typing import TypedDict

from typing_extensions import TypedDict # Use typing_extensions.TypedDict


# Define a TypedDict for the schema
class ChoiceSchema(TypedDict):
    value: str

# Define your enum class
class Choice(enum.Enum):
    PERCUSSION = "打擊樂"
    STRING = "字串"
    WOODWIND = "木管樂器"
    BRASS = "黃銅"
    KEYBOARD = "鍵盤"

model = genai.GenerativeModel("gemini-1.5-pro-latest")

organ = genai.upload_file(path="organ.jpg",
                            display_name="organ")
result = model.generate_content(
    ["這是什麼樂器：",organ],
    generation_config=genai.GenerationConfig(
        response_mime_type="text/x.enum", response_schema=ChoiceSchema
    ),
)


print(result)  # Keyboard

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "\u9019\u4ef6\u6a02\u5668\u662f\u4e00\u53f0**\u7ba1\u98a8\u7434**\u3002"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
 

### 您也可以將架構作為 JSON 傳遞：

In [32]:
model = genai.GenerativeModel("gemini-1.5-pro-latest")

organ = genai.upload_file("organ.jpg")
result = model.generate_content(
    ["請用繁體中文回答這是什麼樂器：", organ],
    generation_config=genai.GenerationConfig(
        response_mime_type="text/x.enum",
        response_schema={
            "type": "STRING",
            "enum": ["打擊樂", "字串", "木管樂器", "黃銅", "鍵盤"],
        },
    ),
)
print(result)  # Keyboard

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": " \u9375\u76e4"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
      "usage_metadata": {
        "prompt_token_c



### 向模型詢問食譜標題列表，並使用列舉Grade為每個標題提供評份等級



In [34]:
!pip install pydantic

In [45]:
import enum
from typing import List, TypedDict
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

class Grade(enum.Enum):
    A_PLUS = "a+"
    A = "a"
    B = "b"
    C = "c"
    D = "d"
    F = "f"

class Recipe(TypedDict):
    grade: Grade
    recipe_name: str

model = genai.GenerativeModel("gemini-1.5-pro-latest")

result = model.generate_content(
    "列出大約 10 種中餐食譜，根據受歡迎程度對它們進行評分",
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema={
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "grade": {"type": "string", "enum": [e.value for e in Grade]},
                    "recipe_name": {"type": "string"},
                },
            },
        }
    ),
    request_options={"timeout": 600},
)
print(result)  # [{"grade": "a+", "recipe_name": "Mapo Tofu"}, ...]

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "[{\"grade\": \"a+\", \"recipe_name\": \"Kung Pao Chicken\"}, {\"grade\": \"a+\", \"recipe_name\": \"Mapo Tofu\"}, {\"grade\": \"a\", \"recipe_name\": \"Chow Mein\"}, {\"grade\": \"a\", \"recipe_name\": \"Sweet and Sour Pork\"}, {\"grade\": \"a\", \"recipe_name\": \"Dumplings\"}, {\"grade\": \"a\", \"recipe_name\": \"Spring Rolls\"}, {\"grade\": \"b\", \"recipe_name\": \"Peking Duck\"}, {\"grade\": \"b\", \"recipe_name\": \"Wontons\"}, {\"grade\": \"b\", \"recipe_name\": \"Fried Rice\"}, {\"grade\": \"b\", \"recipe_name\": \"Lo Mein\"}] "
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_E